After the first NN was succesfully created I wanted to make some additions to it. So I let the first notebook be a "backup" and here we go!

The task is to add barriers that the network has to overcome. Here we go!

Boundry incorrect

Bad fitness

In [1]:
display_width = 800
display_height = 600
import numpy as np 
class NoWeightsError(ValueError):
    def __init__(self, arg):
        self.strerror = arg
        self.args = {arg}
    
class neuron():
    
    def __init__(self):
        """
        value has no meaning at this point. bias is 0 - 0.1 and at neuralnet class, weights are set between -10 and 10
        copy() returns neuron with same values for all
        mutate() is in place and assumes network has been created; adjusts weights
        """
        
        self.value = float(np.random.rand(1))

        self.bias = (float(np.random.rand(1))*2-1)/10
        self.weights = []
        
    def __str__(self):
        return str(self.value) + " " + str(self.bias) + " " + str(self.weights) + "\n"
    
    def __eq__(self, neuron):
        return self.bias == neuron.bias and self.weights == neuron.weights
    
    def copy(self):
        
        copycat = neuron()
        copycat.value = self.value
        copycat.bias = self.bias
        copycat.weights = list(self.weights)
        return copycat
        
    def mutate(self, mutation_rate=1):
        
        self.bias += mutation_rate*(float(np.random.rand(1))*2-1)/100

        if len(self.weights) == 0:
            raise NoWeightsError("Neuron does not have weights!")
        
        for i in range(0, len(self.weights)):
            #if float(np.random.rand(1)) < 1/len(self.weights):
            self.weights[i] += mutation_rate*(float(np.random.rand(1))*2-1)

class layers():
    """
    copy() returns different layer object
    mutate() is in place
    """
    def __init__(self, num_layers):
        self.layer = []
        
        for i in range(0,num_layers):
            n = neuron()
            self.layer.append(n)
        
    def __str__(self):
        for i in self.layer:
            print(i)
        return ""
    
    def copy(self):
        copycat = layers(len(self.layer))
        copycat.layer = [neuron.copy() for neuron in self.layer]
        return copycat
    
    def mutate(self, mutation_rate=1):
        for index, neuron in enumerate(self.layer):
            try:
                neuron.mutate(mutation_rate)
            except NoWeightsError:
                if len(self.layer) != 6:
                    print("NoWeightsError raised")

    
class neuralnet():
    """
    copy() returns new neuralnet object. DOES NOT COPY THE TRACKER VARIABLES
    mutate() acts in place
    """
    def sigmoid(self, x):
        sigm = 1 / (1 + np.exp(-x))
        return sigm
    
    def __init__(self):
        self.list_layers = [layers(6), layers(8), layers(8), layers(4)]
        
        self.sxpos = display_width/2-50
        self.sypos = display_height - 100
        
        self.sxdif = 0
        self.sydif = 0
        
        self.netxpass = 0
        self.netypass = 0
        
        self.alive = True
        
        self.fitness = 0
    
        for index in range(0, len(self.list_layers)):
            if index == 0:
                continue
            
            for neuron in self.list_layers[index].layer:
                
                #Set weights for neuron
                for i in range(0,len(self.list_layers[index-1].layer)):
                    neuron.weights.append((float(np.random.rand(1))*2-1)*10)
                    
    def reset(self):
        self.sxpos = display_width/2-50
        self.sypos = display_height - 100
        
        self.sxdif = 0
        self.sydif = 0
        
        self.netxpass = 0
        self.netypass = 0
        
        self.alive = True
        
        self.fitness = 0
        
    def pass_values(self, input_tuple):
        
        if self.alive:
            
            for neuron, val in zip(self.list_layers[0].layer, input_tuple):
                self.list_layers[0].layer[self.list_layers[0].layer.index(neuron)].value = val
            
            #self.list_layers[0].layer[0].value, self.list_layers[0].layer[1].value = input_tuple

            for index in range(0, len(self.list_layers)):
                if index == 0:
                    continue

                for neuron in self.list_layers[index].layer:

                    #Set value for neuron
                    neuron.value = 0
                    for i in range(0,len(self.list_layers[index-1].layer)):
                        for index2, wieght in enumerate(neuron.weights):
                            neuron.value += wieght * self.list_layers[index-1].layer[index2].value + neuron.bias
                        neuron.value = self.sigmoid(neuron.value)

            #Find the output neurons
            final_layer = self.list_layers[3].layer

            return (final_layer[0].value, final_layer[1].value, final_layer[2].value, final_layer[3].value)
        elif not self.alive:
            return (0,0,0,0)
    
    def __str__(self):
        
        for layer in self.list_layers:
            print("NEW LAYER\n")
            print(layer)
        return ""
    
    def __eq__(self, net):
        neuron_check = True
        for layer, otherlayer in zip(self.list_layers, net.list_layers):
            for neuron, otherneuron in zip(layer.layer, otherlayer.layer):
                if neuron == otherneuron:
                    continue
                neuron_check = False
                break
            
            if neuron_check:
                continue
            break
        return neuron_check
    
    def show_output(self): 
        print(self.list_layers[-1])    
        
    def train(self):
        # Suppossed to backpropogate
        pass #Holy cow how will I do this
    
    def copy(self):
        copycat = neuralnet()
        copycat.list_layers = [layers.copy() for layers in self.list_layers]
        return copycat
    
    def mutate(self, mutation_rate=1):
        for layer in self.list_layers:
            layer.mutate(mutation_rate)

        

class population():
    
    def __init__(self, num_nets):
        
        self.all_nets = []
        for i in range (0,num_nets):
            self.all_nets.append(neuralnet())
    
    def __eq__(self, pop):
        general_check = True
        for net, othernet in zip(self.all_nets, pop.all_nets):
            if net == othernet:
                continue
            general_check = False
            break
        return general_check
            
    def copy(self):
        
        copycat = population(len(self.all_nets))
        copycat.all_nets = [net.copy() for net in self.all_nets] 
        return copycat
        
    def reset(self):
        
        for net in self.all_nets:       
            net.reset()
            
    def mutate(self, top_performing_net, mutation_rate =1):
        # Maybe calculate best net by itself? (using best())
        
        num_nets = len(self.all_nets)
        
        self.all_nets = []
        top_performing_net.reset()
        self.all_nets.append(top_performing_net)
        for i in range(0, 7):
            self.all_nets.append(neuralnet())

        for i in range(0, num_nets-8):
            copy = top_performing_net.copy()
            copy.mutate()
            self.all_nets.append(copy)
        
    
    def best(self):
        best_net = neuralnet()
        for net in self.all_nets:
            if net.fitness > best_net.fitness:
                best_net = net
        
        return best_net
                

In [2]:
p = population(10)
for net in p.all_nets:
    net.pass_values(tuple(np.random.rand(225)))

In [3]:
import pygame, sys, time
from pygame.locals import *

pygame.init()
black = (0,0,0)
white = (255,255,255)
red = (255,0,0)
blue = (0,0,255)
green = (0,255,0)
navy_blue = (0,60,128)

display_width = 800
display_height = 600
gameDisplay = pygame.display.set_mode((display_width, display_height))

pygame.display.set_caption('FIRST PYGAME NN ATTEMPT')
clock = pygame.time.Clock()

def escape(): 
    pygame.quit()
    sys.exit()

def crash():
    display_message("You crashed!")
    
def win():
    display_message("You win!")
    
def solved():
    display_message("Task has been solved")
    
def newpop():
    display_message("New Gen Start")
    
def resetpop():
    display_message("Restarting Generation")
    
def display_message(text):
    largeText = pygame.font.Font('freesansbold.ttf',50)
    TextSurf, TextRect = text_objects(text, largeText)
    TextRect.center = ((display_width/2),(display_height/2))
    gameDisplay.blit(TextSurf, TextRect)
    pygame.display.update()
    time.sleep(1)
    
def display_variables(text, posx, posy):
    textfont = pygame.font.Font('freesansbold.ttf', 12)
    TextSurf, TextRect = text_objects(text, textfont)
    TextRect.center = (posx, posy)
    gameDisplay.blit(TextSurf, TextRect)


def text_objects(text, font):
    textSurface = font.render(text, True, white)
    return textSurface, textSurface.get_rect()    

def check_boundries(sxcoor, sycoor, swidth, sheight, display_height, display_width):
    
    sxcoor, sycoor = return_middle(sxcoor, sycoor, swidth, sheight)
    
    if sxcoor > 0 and sxcoor < display_width and sycoor > 0 and sycoor < display_height :
        return True
    return False
    
def check_win(sxcoor, sycoor, swidth, sheight, gsxcoor, gsycoor):
    
    sxcoor, sycoor = return_middle(sxcoor, sycoor, swidth, sheight)
    gsxcoor, gsycoor = return_middle(gsxcoor, gsycoor, swidth, sheight)

    
    if abs(gsxcoor-sxcoor) < swidth and abs(gsycoor - sycoor) < swidth:
        return True
    return False

def check_safe(sxcoor, sycoor, swidth, sheight, bsxcoor, bsycoor, bswidth, bsheight):
    
    
    if abs(bsxcoor-sxcoor) < abs(swidth/2+bswidth/2) and abs(bsycoor-sycoor) < (sheight/2+bsheight/2):
        return True
    return False

def return_middle(xcoor, ycoor, width, height):
    xcoor -= width/2
    ycoor -= height/2
    
    return xcoor,ycoor

    
def square(color, xcoor, ycoor, width, height):
    
    xcoor, ycoor = return_middle(xcoor, ycoor, width, height)
    
    pygame.draw.rect(gameDisplay, color, [xcoor, ycoor, width, height])

#Initializitations
population_number = 25
net_pop = population(population_number)
saved_net_pop = population(population_number)


def game_loop():
    
    #Tracker variables
    num_winners = 0
    all_dead = False
    num_AI_left = population_number
    seconds = 0
    generation_number = 1
    top_performing_net = neuralnet()
    improvement = 0

    #Constant variables
    swidth = 20
    sheight = 20 
    gsxcoor = display_width/2+100
    gsycoor = 50
    start_ticks=pygame.time.get_ticks() #starter tick for seconds
    mutation_rate = 1
    bsxcoor = display_width/2 + 50
    bsycoor = display_height/2
    bswidth = 300
    bsheight = 50
    
    # Declare global variables
    global saved_net_pop
    global net_pop
    
    while True: # main game loop => Human observation
        
        #Human Operations start here
        for event in pygame.event.get():

            
            if event.type == QUIT:
                escape()
                
            if event.type == pygame.KEYDOWN:  
                
                if event.key == pygame.K_SPACE:
                    escape()
                    
                if event.key == pygame.K_s:
                    saved_net_pop = net_pop.copy()
                    saved_net_pop.reset()
                    escape()
                
                if event.key == pygame.K_r:
                    resetpop()
                    net_pop.reset()
                    
                    num_winners = 0
                    num_AI_left = population_number
                    start_ticks = pygame.time.get_ticks()
                    
                if event.key == pygame.K_UP:
                    mutation_rate += 0.1
                    
                if event.key == pygame.K_DOWN:
                    mutation_rate -= 0.1
                
                if event.key == pygame.K_f:
                    for net in net_pop.all_nets:
                        if net.alive:
                            net.alive = False
                            net.fitness = 100/(abs(net.sxpos - gsxcoor) + abs(net.sypos - gsycoor))

        
        #AI Operations start here
        for net in net_pop.all_nets:
            
            net.netxpass = gsxcoor - net.sxpos
            net.netypass = gsycoor - net.sypos
                        
            output_tuple = net.pass_values((net.netxpass, net.netypass, bsxcoor, bsycoor, bswidth, bsheight))
            
            # 0 = left; 1 = up; 2 = right; 3 = down
            if output_tuple[0] == max(output_tuple):
                net.sydif -= 5
            if output_tuple[1] == max(output_tuple):
                net.sxdif -= 5
            if output_tuple[2] == max(output_tuple):
                net.sxdif += 5
            if output_tuple[3] == max(output_tuple):
                net.sydif += 5

            net.sxpos += net.sxdif
            net.sypos += net.sydif
        
            net.sxdif = 0
            net.sydif = 0
    
            #Apply Logic to all AI     ++++++ ADD REWARD FUNCTIONALITY
            if net.alive:
                if not check_boundries(net.sxpos, net.sypos, swidth, sheight, display_height, display_width):
                    net.alive = False
                    num_AI_left -= 1
                    net.fitness = 100/(abs(net.sxpos - gsxcoor) + abs(net.sypos - gsycoor))
                    if net.sypos < bsycoor:
                        net.fitness += 0.2
                    
                if check_safe(net.sxpos, net.sypos, swidth, sheight, bsxcoor, bsycoor, bswidth, bsheight):
                    net.alive = False
                    num_AI_left -= 1
                    net.fitness = 100/(abs(net.sxpos - gsxcoor) + abs(net.sypos - gsycoor))
                
                if check_win(net.sxpos, net.sypos, swidth, sheight, gsxcoor, gsycoor):
                    num_winners +=1
                    net.alive = False
                    net.fitness = 100 - seconds
                    if net.sypos < bsycoor:
                        net.fitness += 0.2
                    
        
        #Time limit
        seconds=(pygame.time.get_ticks()-start_ticks)/1000 #calculate how many seconds
        if seconds > 15:
            for net in net_pop.all_nets:
                if net.alive:
                    net.alive = False
                    net.fitness = 100/(abs(net.sxpos - gsxcoor) + abs(net.sypos - gsycoor))
        
        #Visuals
        gameDisplay.fill(black)         
        #display all variables
        display_variables("Seconds: " + str(seconds), display_width-100, 15)
        display_variables("Num AI Left: " + str(num_AI_left), display_width-100, 30)
        display_variables("Num winners: " + str(num_winners), display_width-100, 45)
        display_variables("Highest Fitness: " + "{:.5f}".format(net_pop.best().fitness), display_width-100, 60)
        display_variables("Generation Number: " + str(generation_number), display_width-100, 75)
        display_variables("Mutation Rate : " + "{:.1f}".format(mutation_rate), display_width-100, 90)
        display_variables("Improvement : " + "{:.5f}".format(improvement), display_width-100, 105)

        
        for net in net_pop.all_nets:
            if top_performing_net == net:
                continue
            elif net.alive:
                square(blue, net.sxpos, net.sypos, swidth, sheight) 
                
        for net in net_pop.all_nets:
            if top_performing_net == net:
                square(green, net.sxpos, net.sypos, swidth, sheight) 
        
        square(navy_blue, bsxcoor, bsycoor, bswidth, bsheight)
        square(red, gsxcoor, gsycoor, swidth, sheight)

        #Check to see if population is finished
        for net in net_pop.all_nets:
            if net.alive:
                all_dead = False
                break
            all_dead = True
        
        #Check if the solution has been found
        if top_performing_net.fitness > 50:
            print(top_performing_net.fitness)
            solved()
            escape()
            break
        
        #Start next process if population had been tested
        if all_dead:
            newpop()
            
            improvement = net_pop.best().fitness - top_performing_net.fitness
            # CHANGE TO MUTATED POPULATION
            top_performing_net = net_pop.best()
            net_pop.mutate(top_performing_net, mutation_rate)
            
            generation_number += 1
            num_winners = 0
            num_AI_left = len(net_pop.all_nets)
            start_ticks = pygame.time.get_ticks()        
        
        #Make movement behavior regular and paced
    
        pygame.display.update()
        clock.tick(60)
        

game_loop()

pygame 2.0.1 (SDL 2.0.14, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\megat\AppData\Local\Temp\ipykernel_25852\71866061.py:85: RuntimeWarning: overflow encountered in exp
  sigm = 1 / (1 + np.exp(-x))


SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
cool_pop=saved_net_pop

In [23]:
sol_pop=cool_pop